<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/ch13_geometryIntegration_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 13 (Geometry of integration)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import Math

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 13.1: Define bounds and $\Delta$x

In [ ]:
# specify bounds
a = -.5
b = np.pi

# create deltax
n = 14
deltax = (b-a)/n

# define partition points
breakPointsL = [ a+deltax*i for i in range(n+1) ]
breakPoints = np.linspace(a,b,n+1)

# confirmation
print(f'delta-x: {deltax:.3f}')
print(f'x_p dist.: {breakPoints[1]-breakPoints[0]:.3f}')

print('')
print('Breakpoints from list comprehension:')
print([ f'{i:.3f}' for i in breakPointsL])

print('')
print('Breakpoints from np.linspace:')
print([ f'{i:.3f}' for i in breakPoints])

# Exercise 13.2: Draw the function and area

In [ ]:
# function for the function
def fx(x): return np.exp(x)/10 + np.cos(x)

# x-axis spacing
xx = np.linspace(a-.5,b+.5,809)

# show the function, bounds, area, and breakpoints
plt.figure(figsize=(10,4))
plt.plot(xx,fx(xx),'k',linewidth=2,label=r'$f(x) = \cos(x) + e^{x}/10$')

plt.axvline(a,color=[.4,.4,.4],linestyle='--',label=f'a = {a:.2f}')
plt.axvline(b,color=[.4,.4,.4],linestyle=':',label=f'b = {b:.2f}')
plt.axhline(0,color=[.8,.8,.8],linestyle='--',label='y = 0')

# shaded region for area calculation
xVals2plot = (xx>a) & (xx<b)
plt.fill_between(xx[xVals2plot],fx(xx[xVals2plot]),color='k',alpha=.2,edgecolor=None,label='Area')

# breakpoints as x-axis tick marks
for bp in breakPoints:
  plt.plot([bp,bp],[-.05,.05],'k')


plt.legend(loc='upper center')
plt.gca().set(xlabel='x',ylabel='y=f(x)',xlim=xx[[0,-1]],ylim=[-.05,1.5])

plt.tight_layout()
plt.savefig('geoApprox_ex2.png')
plt.show()

# Exercise 13.3: Compute Riemann sums and compare with the integral

In [ ]:
# Riemann sums

# left rule
area_left = np.sum(fx( breakPoints[:-1] )) * deltax

# right rule
area_right = np.sum(fx( breakPoints[1:] )) * deltax

# midpoint rule
area_midpoint = np.sum(fx( (breakPoints[:-1]+breakPoints[1:])/2 )) * deltax

In [ ]:
# true integral using sympy
t = sym.symbols('t')
area_analytic = sym.integrate( sym.exp(t)/10 + sym.cos(t),(t,a,b))

# print all results
print(f'    Using left rule: {area_left:.8f}')
print(f'   Using right rule: {area_right:.8f}')
print(f'Using midpoint rule: {area_midpoint:.8f}')
print(f'     Sympy integral: {area_analytic:.8f}')

# Exercise 13.4: Visualize the partitions

In [ ]:
# show the function, bounds, area, and breakpoints
_,axs = plt.subplots(1,3,figsize=(16,4))


# same for all plots
for ax in axs:
  ax.plot(xx,fx(xx),'k',linewidth=2,label=r'$f(x) = \cos(x) + e^{x}/10$')
  ax.axvline(a,color=[.4,.4,.4],linestyle='--',label=f'a = {a:.2f}')
  ax.axvline(b,color=[.4,.4,.4],linestyle=':',label=f'b = {b:.2f}')
  # ax.legend(fontsize=11)
  ax.set(xlabel='$x$',ylabel='$y=f(x)$',xlim=xx[[0,-1]],ylim=[0,1.5])


# now for the bars
for i in range(n):

  # bars for left rule
  bp = breakPoints[i]
  axs[0].fill_between([bp,bp+deltax],[fx(bp),fx(bp)],color='k',alpha=.2)

  # bars for right rule
  axs[1].fill_between([bp,bp+deltax],[fx(bp+deltax),fx(bp+deltax)],color='k',alpha=.2)

  # bars for midpoint rule
  bp += deltax/2 # shift breakpoint by deltax/2
  axs[2].fill_between([bp-deltax/2,bp+deltax/2],[fx(bp),fx(bp)],color='k',alpha=.2)



# plot titles
axs[0].set_title(fr'$\bf{{A}}$)  Left rule: area={area_left:.4f}')
axs[1].set_title(fr'$\bf{{B}}$)  Right rule: area={area_right:.4f}')
axs[2].set_title(fr'$\bf{{C}}$)  Midpoint rule: area={area_midpoint:.4f}')

plt.tight_layout()
plt.savefig('geoApprox_rules1.png')
plt.show()

# Exercise 13.5: Demonstrate convergence with shrinking $\Delta$x

In [ ]:
Ns = np.arange(5,191)
areas = np.zeros((len(Ns),3))

for i,n in enumerate(Ns):
  deltax = (b-a)/n
  breakPoints = np.linspace(a,b,n+1)

  areas[i,0] = np.sum(fx( breakPoints[:-1] )) * deltax
  areas[i,1] = np.sum(fx( breakPoints[1:] )) * deltax
  areas[i,2] = np.sum(fx( (breakPoints[:-1]+breakPoints[1:])/2 )) * deltax


# visualization
plt.figure(figsize=(8,4))
plt.plot(Ns,areas[:,0],color=[0,0,0],linewidth=3,label='Left')
plt.plot(Ns,areas[:,1],color=[.3,.3,.3],linestyle='-.',linewidth=3,label='Right')
plt.plot(Ns,areas[:,2],color=[.6,.6,.6],linestyle=':',linewidth=3,label='Midpoint')
plt.axhline(area_analytic,color='k',linestyle='--',label='Analytic')
plt.legend()

plt.gca().set(xlim=Ns[[0,-1]],xlabel='Number of partitions',ylabel='Area (a.u.)')

plt.tight_layout()
plt.savefig('geoApprox_ex5.png')
plt.show()

# Exercise 13.6: Trapezoid rule

In [ ]:
# function for the function
def fu(u):
  return u**4 - .5

In [ ]:
# partitioning parameters
n = 6
a = -.5
b = 1
deltau = (b-a)/n

# partition breakpoints
breakPoints = [ a+deltau*i for i in range(n+1) ]


# plot the function
_,axs = plt.subplots(1,figsize=(7,4))

# plot the function
uu = np.linspace(a-.1,b+.1,301)
axs.plot(uu,fu(uu),'k')


# initialize areas
areaNet = 0
areaTot = 0

# plot rectangles
for bp in breakPoints[:-1]:

  # find the function value at both upper edges
  yL = fu(bp)
  yR = fu(bp+deltau)

  # draw the rectangle
  faCo=[.9,.5,.5] if yL+yR<0 else [.5,.9,.4]
  axs.fill_between([bp,bp+deltau],[yL,yR],edgecolor='k',facecolor=faCo,alpha=.5)

  # sum the area
  areaNet += ( (yL+yR)/2 ) * deltau
  areaTot += np.abs( (yL+yR)/2 ) * deltau

# set the labels
axs.set(xlabel='u',ylabel=r'$y = u^4-.5$')
axs.set_title(r'net area = %.3f, total area = %.3f, $\Delta$u=%g' %(areaNet,areaTot,deltau),wrap=True,loc='center')

# finalize
plt.axhline(0,color='gray',linestyle='--',zorder=-7)
plt.axvline(a,color='gray',linestyle='--',zorder=-7)
plt.axvline(b,color='gray',linestyle='--',zorder=-7)
plt.xlim(uu[[0,-1]])

plt.tight_layout()
plt.savefig('geoApprox_ex6.png')
plt.show()

# Exercise 13.7: net vs. total area in sympy

In [ ]:
# symbolic variable
tau = sym.symbols('tau')

# function defs
funs = [None]*3
funs[0] = sym.cos(2*sym.pi*tau) * sym.exp(-tau)
funs[1] = tau**3 - 1
funs[2] = sym.exp(-sym.Abs(tau))

# x-axis grid
xx = np.linspace(-1,1.5,801)
a,b = -.5, 1
xx4patch = (xx>a) & (xx<b)


# setup figure
_,axs = plt.subplots(1,3,figsize=(12,4))

# loop over functions
for f,ax,funL in zip(funs,axs,'fgh'):

  # evaluate
  f_n = np.array([f.subs(tau,i) for i in xx],dtype=float)

  # plot
  ax.plot(xx,f_n,'k',label=r'$%s(\tau) = %s$'%(funL,sym.latex(f)))
  ax.axhline(0,color=[.7,.7,.7],linestyle='--',zorder=-4)
  ax.fill_between(xx[xx4patch],f_n[xx4patch],color='k',alpha=.2)

  # calculate area
  netArea = sym.integrate(        f ,(tau,a,b))
  totArea = sym.integrate(sym.Abs(f),(tau,a,b))

  ax.set(xlim=xx[[0,-1]],xlabel='$\\tau$',ylabel=r'$y = f(\tau)$')
  ax.set_title(f'Net: {netArea:.4f}\nTotal: {totArea:.4f}',loc='center')
  ax.legend()


plt.tight_layout()
plt.savefig('geoApprox_NetVsTotal.png')
plt.show()

# Exercise 13.8: Net and total areas of cosine

In [ ]:
x = sym.symbols('x')

# function and integration bounds
f = sym.cos(x)
a = sym.pi/2
b = 5*sym.pi/2

# find the x value for splitting
splitx = sym.solveset(f,x,domain=sym.Interval.open(a,b)).args[0]
splitx

In [ ]:
# the function and plotting bounds
f_lam = sym.lambdify(x,f)
xx = np.linspace(float(a)-1,float(b)+1,179)

# plot the function
plt.figure(figsize=(10,4))
plt.plot(xx,f_lam(xx),'k')

# plot the lines
plt.axhline(0,color=[.7,.7,.7],zorder=-3,linestyle='--')
plt.axvline(float(a),color=[.7,.7,.7],linestyle=':')
plt.axvline(float(b),color=[.7,.7,.7],linestyle=':')
plt.axvline(float(splitx),color=[.7,.7,.7],linestyle=':')

# and the areas
x2plot = xx[(xx>float(a)) & (xx<float(splitx))]
plt.fill_between(x2plot,f_lam(x2plot),zorder=-5,hatch='-',color='r',facecolor='none',alpha=.4)
x2plot = xx[(xx>float(splitx)) & (xx<float(b))]
plt.fill_between(x2plot,f_lam(x2plot),zorder=-5,hatch='+',color='g',facecolor='none',alpha=.4)

# adjustments
plt.gca().set(xlabel='$x$',ylabel='$y=f(x)$',ylim=[-1.1,1.1],xlim=xx[[0,-1]])

plt.tight_layout()
plt.savefig('geoApprox_ex8.png')
plt.show()

In [ ]:
## net area
area = sym.integrate(f,(x,a,b))

## total area by absolute value
areaTabs = sym.integrate(sym.Abs(f),(x,a,b))

## total area by splitting
area1 = sym.Abs( sym.integrate(f,(x,a,splitx)) )
area2 = sym.Abs( sym.integrate(f,(x,splitx,b)) )
areaTsplit = area1 + area2

# print the results
display(Math('\\text{Net: } \int_{%s}^{%s} %s \,dx = %s'%(sym.latex(a),sym.latex(b),sym.latex(f),sym.latex(area))))
display(Math('\\text{Total: } \int_{%s}^{%s} \\left| %s \\right| \,dx = %s'%(sym.latex(a),sym.latex(b),sym.latex(f),sym.latex(areaTabs))))
display(Math('\\text{Total: } \\left| \int_{%s}^{%s} %s\,dx \\right| + \\left| \int_{%s}^{%s} %s\,dx \\right| = %s+%s = %s'%(sym.latex(a),sym.latex(splitx),sym.latex(f),
                                                                                                                             sym.latex(splitx),sym.latex(b),sym.latex(f),
                                                                                                                             sym.latex(sym.latex(area1)),sym.latex(area2),
                                                                                                                             sym.latex(areaTsplit) )))

# Exercise 13.9

In [ ]:
# sympy expression for function
x = sym.symbols('x')
f = x**2 - 2*x
# f = sym.cos(2*sym.pi*x)

# vector of values of b
bs = np.linspace(0,3.5,30)

# initialize
netAreas = np.zeros(len(bs))
totAreas = np.zeros(len(bs))

# calculate the net and total areas
for idx in range(len(bs)):
  netAreas[idx] = sym.integrate(        f ,(x,-1/2,bs[idx]))
  totAreas[idx] = sym.integrate(sym.Abs(f),(x,-1/2,bs[idx]))

In [ ]:
_,axs = plt.subplots(2,1,figsize=(10,6))

# discretized function values
xx = np.linspace(-1/2,bs[-1],801)
yy = np.array([f.subs(x,i) for i in xx],dtype=float)

# draw the function
axs[0].plot(xx,yy,'k')
axs[0].axhline(0,linestyle=':',color=[.8,.8,.8],zorder=-4)
axs[0].set(xlabel='x',ylabel=r'$y=f(x)$',xlim=[xx[0]-.05,xx[-1]+.05],title=r'$\bf{A}$)  Function')

# and the areas
axs[1].plot(bs,netAreas,'ks-',markersize=8,label='Net area')
axs[1].plot(bs,totAreas,'o--',markersize=8,color=[.4,.4,.4],label='Total area')
axs[1].axhline(0,linestyle=':',color=[.8,.8,.8],zorder=-4)
axs[1].set(xlabel='Upper bound of definite integral',ylabel='Area',xlim=[xx[0]-.05,xx[-1]+.05],title=r'$\bf{B}$)  Areas')
axs[1].legend()

plt.tight_layout()
plt.savefig('geoApprox_ex9.png')
plt.show()

In [ ]:
# calculate and print the antiderivatives
antiderivF = sym.integrate(f,x)
antiderivFabs = sym.integrate(sym.Abs(f),x)

display(Math('%s \;=\; %s' %(sym.latex(sym.Integral(f,x)),sym.latex(antiderivF))))
print('')
display(Math('%s \;=\; %s' %(sym.latex(sym.Integral(sym.Abs(f),x)),sym.latex(antiderivFabs))))

In [ ]:
sym.integrate(sym.Abs(f),(x,0,1))#.evalf()

In [ ]:
# NOTE: When using sym.cos(2*sym.pi*x), the total area oscillates, which is incorrect.
# Sympy struggles with the periodic nature of this function. This is an illustration of
# why it's often best to numerically evaluate a function using high resolution, then
# approximate the definite integral using numerical methods.

# Exercise 13.10: Riemann approximation

In [ ]:
# function for the function
def fx(u):
  return u**2 + np.cos(2*np.pi*u)/5

In [ ]:
# create deltax
n = 12
a = 0
b = 1
deltax = (b-a)/n

breakPoints = [ a+deltax*i for i in range(n+1) ]


# plot the function
_,axs = plt.subplots(1,figsize=(10,4))

# plot the function
xx = np.linspace(a-.1,b+.1,301)
axs.plot(xx,fx(xx),'k',label=r'$f(x) = x^2+\cos(2\pi x)/5$')

# initialize area
riemann_approx = 0

# plot rectangles
for i in range(n):

  # compute the function value at midpoint
  bp = breakPoints[i] + deltax/2 # shift breakpoint by deltax/2
  y  = fx(bp)

  # draw the rectangle
  axs.fill_between([breakPoints[i],breakPoints[i+1]],[y,y],color='k',edgecolor='k',alpha=1-i/n)

  # sum the area
  riemann_approx += y * deltax

# set the labels (*after* the for-loop)
axs.set(xlim=xx[[0,-1]],xlabel='x',ylabel=r'$y = f(x)$')
axs.set_title(r'Approximate net area = %.3f, $\Delta$x=%.4f' %(riemann_approx,deltax),loc='center')
axs.legend()

# finalize
axs.axhline(0,color='gray',linewidth=1,linestyle='--',zorder=-4)
axs.axvline(a,color='gray',linewidth=1,linestyle='--',zorder=-4)
axs.axvline(b,color='gray',linewidth=1,linestyle='--',zorder=-4)

plt.tight_layout()
plt.savefig('geoApprox_ex10.png')
plt.show()

# Exercise 13.11: Lebesgue approximation

In [ ]:
#### subgoal 1: partition the range

# fine partitioning of the x-axis (domain)
domain_n = 1000
domainPoints = np.linspace(a,b,domain_n)
deltax = (b-a)/domain_n

# evaluate the function at those points
fx_values = fx(domainPoints)

# determine the range of the function in this domain
min_val, max_val = np.min(fx_values), np.max(fx_values)

# define boundaries for range of fx
yPartitions = np.linspace(min_val,max_val,n+1)
deltay = yPartitions[1]-yPartitions[0]

# initialize Lebesgue approximation
lebesgue_approx = 0




# plot the function
_,axs = plt.subplots(1,figsize=(10,4))

# plot the function
xx = np.linspace(a-.1,b+.1,301)
axs.plot(xx,fx(xx),'k',label=r'$f(x) = x^2+\cos(2\pi x)/5$')


for i in range(n):


  #### subgoal 2: measure the area for this partition

  # find points where the function is within the current partition
  in_partition = (fx_values >= yPartitions[i]) & (fx_values <= yPartitions[i+1])

  # measure the "size" of that set
  measure = np.sum(in_partition) * deltax

  # The average function value on this set
  average_value = (yPartitions[i] + yPartitions[i+1]) / 2

  # sum this set to the integral approximation
  lebesgue_approx += average_value * measure



  #### subgoal 3: draw the rectangles

  # find the contiguous groups in in_partition
  in_partition_diff = np.diff(in_partition.astype(int))
  group_starts = np.where(in_partition_diff == 1)[0] + 1  # start points of groups
  group_ends = np.where(in_partition_diff == -1)[0]       # end points of groups


  # in case a group starts/ends at the integration bounds
  if in_partition[0]:
    group_starts = np.insert(group_starts, 0, 0)
  if in_partition[-1]:
    group_ends = np.append(group_ends, len(in_partition) - 1)


  # loop over groups and draw rectangles for each
  for start,end in zip(group_starts, group_ends):

    # visualization option "a"
    x1,x2 = domainPoints[start], domainPoints[end]
    y1,y2 = yPartitions[i], yPartitions[i+1]

    # visualization option "b"
    x1,x2 = domainPoints[start], domainPoints[end]
    y1,y2 = 0, yPartitions[i+1]

    # visualization option "c" (used in the book figure)
    x1 = a if fx(domainPoints[end])<fx(domainPoints[start]) else domainPoints[start]
    x2 = b if fx(domainPoints[end])>fx(domainPoints[start]) else domainPoints[end]
    y1,y2 = yPartitions[i],yPartitions[i+1]

    # draw the patch
    axs.fill_between([x1,x2],y1,y2,color='k',edgecolor='k',alpha=1-i/n)



# set the labels (after the for-loop)
axs.set(xlim=xx[[0,-1]],xlabel='x',ylabel=r'$y = f(x)$',yticks=yPartitions[::2])
axs.set_title(r'Approximate net area = %.3f, $\Delta$y=%.4f' %(lebesgue_approx,deltay),loc='center')
axs.legend()

# finalize
axs.axhline(0,color='gray',linewidth=1,linestyle='--',zorder=-4)
axs.axvline(a,color='gray',linewidth=1,linestyle='--',zorder=-4)
axs.axvline(b,color='gray',linewidth=1,linestyle='--',zorder=-4)

plt.tight_layout()
plt.savefig('geoApprox_ex11.png')
plt.show()

# Exercise 13.12: Compare Riemann, Lebesgue, and analytical

In [ ]:
## define functions

# Riemann
def riemann():
  deltax = (b-a)/n
  breakPoints = [ a+deltax*i for i in range(n+1) ]
  riemann_approx = 0
  for i in range(n):
    bp = breakPoints[i] + deltax/2
    riemann_approx += fx(bp) * deltax
  return riemann_approx



# Lebesgue
def lebesgue():
  domain_n = 1000
  domainPoints = np.linspace(a,b,domain_n)
  deltax = 1/domain_n*(b-a)

  # evaluate the function at those points
  fx_values = fx(domainPoints)

  # determine the range of the function in this domain
  min_val, max_val = np.min(fx_values), np.max(fx_values)

  # define boundaries for range of fx
  yPartitions = np.linspace(min_val,max_val,n+1)
  deltay = yPartitions[1]-yPartitions[0]

  # initialize Lebesgue approximation
  lebesgue_approx = 0

  for i in range(n):
    in_partition = (fx_values >= yPartitions[i]) & (fx_values < yPartitions[i+1])
    measure = np.sum(in_partition) * deltax
    average_value = (yPartitions[i] + yPartitions[i+1]) / 2
    lebesgue_approx += average_value * measure

  return lebesgue_approx

In [ ]:
# number of partitions (same for Riemann and Lebesgue)
n = 12

# run the functions
R = riemann()
L = lebesgue()

# calculate the true integral using sympy
from sympy.abc import x
fx_s = x**2 + sym.cos(2*sym.pi*x)/5
A = sym.integrate(fx_s,(x,a,b))

# report the results!
print(f'Riemann:  {R:.6f}')
print(f'Lebesgue: {L:.6f}')
print(f'Sympy:    {A:.6f}')

In [ ]:
# range of discretizations
Ns = np.arange(4,41)

# initialize results
riemann_results = np.zeros(len(Ns))
lebesgue_results = np.zeros(len(Ns))

# run the experiment!
for i,n in enumerate(Ns):
  riemann_results[i] = riemann()
  lebesgue_results[i] = lebesgue()


# plot the results
_,ax = plt.subplots(1,figsize=(10,4))
ax.plot(Ns,riemann_results,'ks-',markersize=8,label='Riemann')
ax.plot(Ns,lebesgue_results,'s:',markersize=8,color=[.4,.4,.4],label='Lebesgue')
ax.axhline(A,linestyle='--',color='gray',label='Sympy')
ax.set(xlabel='Partitions',xlim=[Ns[0]-.5,Ns[-1]+.5],ylabel='Integral')
ax.legend()

plt.tight_layout()
plt.savefig('geoApprox_ex12.png')
plt.show()